In [1]:
from hey_ac import HeyAC

In [2]:
h = HeyAC(grammar_path = '../grammars/grammar_rom.txt')

In [3]:
texts = [
    'the speed of the fan is too low',
    'fan speed is low',
    'temperature is high',
    'humidity feels too high'
]

In [4]:
for text in texts:
    print(h.prune(text))

{'NN_PROP': ['speed'], 'NN_OBJ': ['fan'], 'VB': [], 'NEG': [], 'JJ': ['low']}
{'NN_PROP': ['speed'], 'NN_OBJ': ['fan'], 'VB': [], 'NEG': [], 'JJ': ['low']}
{'NN_PROP': ['temperature'], 'NN_OBJ': [], 'VB': [], 'NEG': [], 'JJ': ['high']}
{'NN_PROP': ['humidity'], 'NN_OBJ': [], 'VB': [], 'NEG': [], 'JJ': ['high']}
